This R notebook performs the CUSUM test on a structural break in the sea water level data.

As an input the notebook uses Python data frames from the notebook Structural_Breaks_CUSUM_Py.

Another way is running R in the Python Notebook itself, but that is not straightforward as the usual R syntax does not always work.

In [ ]:
library(zoo)
library(sandwich) 
library(lmtest)
library(ggplot2)
library(strucchange)

# Read R data frame from  laad het dataframe in van het gewenste station
df <- readRDS("df_sea_surge.RDS")
print(head(df))

In [ ]:
### DO NOT RUN ###

#Python code for the linear model with surge

def linear_model_with_surge(df):
    """
    Return the fit from the linear model on the given dataset df.
    Wind and season can be enabled and disabled
    """
    y = df['height']
    X = np.c_[
        df['year']-1970, 
        np.cos(2*np.pi*(df['year']-1970)/18.613),
        np.sin(2*np.pi*(df['year']-1970)/18.613),
        df['surge']# * (df['year'] >= 1979)    
    ]
    names = ['Constant', 'Trend', 'Nodal U', 'Nodal V', 'Surge']
        
    X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    fit = model.fit(cov_type='HC0')
    return fit, names

In [ ]:
# Some R code for robust fitting. # The package sandwich, also included in strucchange, consists of 
# several methods to correct for heteroskedasticity and autocorrelation. See also:
# https://stackoverflow.com/questions/4385436/regression-with-heteroskedasticity-corrected-standard-errors

ModelSummary <- function(model, VCOV) {
    print(coeftest(model, vcov. = VCOV))
    print(waldtest(model, vcov = VCOV))
}


In [ ]:
# Extend converted Python data frame with nodal cycle (phase shift and amplitude will be estimated, fixed cycle is known well).
df$trend = df$year - 1970
df$costerm = cos(2*pi*(df$year-1970)/18.613)
df$sinterm = sin(2*pi*(df$year-1970)/18.613)

model <- height ~ trend + costerm + sinterm + surge
sea.lm = lm(model, data = df)
summary(lm)

In [ ]:
plot(df$year,sea.lm$fitted.values)
plot(sea.lm)

In [ ]:
ModelSummary(sea.lm, vcovHC)

In [ ]:
ocus <- efp(model, type="OLS-CUSUM", data=dfa)
bound.ocus <- boundary(ocus, alpha=0.05)
ocus$process <- ts(ocus$process, start=c(begin), end=c(eind))
sctest(ocus)
plot(ocus)